<hr style="border:2px solid #0281c9"> </hr>

<img align="left" alt="ESO Logo" src="http://archive.eso.org/i/esologo.png">  

<div align="center">
  <h1 style="color: #0281c9; font-weight: bold;">ESO Science Archive</h1> 
  <h2 style="color: #0281c9; font-weight: bold;">Jupyter Notebooks</h2>
</div>

<hr style="border:2px solid #0281c9"> </hr>

## Query by source name/position

This notebook demonstrates how to use the astroquery.eso module to search for and download raw HAWK-I near-infrared observations from the ESO archive, specifically targeting Sgr A*, the supermassive black hole at the center of the Milky Way. It guides users through querying both the main ESO archive table and instrument-specific tables, filtering results based on observation parameters, and retrieving raw, unprocessed data for further calibration and analysis, enabling direct access to ESO’s extensive collection of observational datasets.

<hr style="border:2px solid #0281c9"> </hr>

### Importing and basic usage of astroquery.eso 

In this example, we will create an instance of the ESO class using:

```python
from astroquery.eso import Eso # import the ESO module from astroquery
eso = Eso()  # create an instance of the ESO class
```

This instance will be used to demonstrate various queries to the ESO Archive, including searching for raw data and data products from a range of instruments and sources.

In [1]:
from astroquery.eso import Eso # import the ESO module from astroquery

In [2]:
eso_instance = Eso() # create an instance of the ESO class

In [3]:
# DEMO - use the INT server througout the notebook
import os
try:
    _ = os.environ["ESO_TAP_URL"]
except KeyError:
    print("Setting up env var ESO_TAP_URL...")
    os.environ["ESO_TAP_URL"] = "http://dfidev5.hq.eso.org:8123/tap_obs"
    pass

### Performing a Small Cone Search Around Sgr A*

To perform a small cone search of **10 arcseconds** around the source **Sgr A**, we first need to resolve the source name to obtain its **Right Ascension (RA)** and **Declination (Dec)** coordinates. This can be done using an online name resolver such as the ``SkyCoord.from_name()`` method from astropy.coordinates, which allows us to resolve object names into precise RA/Dec coordinates using online astronomical databases. This gives: 

```python
ra = 266.835
dec = -28.38528
```

In coordinate values of degrees in the ICRS (International Celestial Reference System) frame.

In [4]:
from astropy.coordinates import SkyCoord # import the SkyCoord class from the astropy.coordinates module
import astropy.units as u # import the astropy.units module

coords = SkyCoord.from_name('Sgr A*') # create a SkyCoord object from the name of the source
radius = 20 *u.arcmin # set the radius of the search to 20 arcminutes

We use `astroquery.eso` to search for observations of **Sgr A** taken with the **HAWKI** instrument on the **VLT (Very Large Telescope)**.

This search can also be accessed via the following: 

[tap-link]: https://archive.eso.org/tap_obs/sync?REQUEST=doQuery&amp;LANG=ADQL&amp;MAXREC=200&amp;FORMAT=txt&amp;QUERY=SELECT%20*%0Afrom%20ivoa.obscore%0Awhere%20intersects(s_region,%20circle(%27%27,%20266.835,%20-28.38528,%200.1666))=1%0AAND%20instrument_name%20=%20%27HAWKI%27

[sp-link]: https://archive.eso.org/scienceportal/home?data_release_date=*:2025-02-07&pos=266.41682,-29.00782&r=0.016667&fovcorners=266.800868,-29.226927,266.033328,-29.226929,266.034957,-28.787142,266.799236,-28.787141&ins_id=HAWKI&dp_type=IMAGE&sort=dist,-fov,-obs_date&s=P%2fDSS2%2fcolor&f=0.669626&fc=266.799236,-28.787141&cs=J2000&av=true&ac=false&c=9,10,11,12,13,15,16,17,18,19,20,21&ta=RES&dts=true&sdtm=%7b%22IMAGE%22%3atrue%7d&at=266.41682,-29.00782&sr=i

- **TAP** - [see here][tap-link]
- **ESO Science Portal** [see here](sp-link)

In [5]:
instrument = 'HAWKI'       # set the instrument to HAWKI
ra = coords.ra             # get the right ascension of the source
dec = coords.dec           # get the declination of the source
radius = radius.to('deg')  # convert the radius to degrees

table_instrument = eso_instance.query_instrument(instrument, ra=ra.value, dec=dec.value, radius=radius.value) # query
table_main = eso_instance.query_main(instrument, ra=ra.value, dec=dec.value, radius=radius.value) # query 

/Users/jcarmona/miniconda3/envs/py39/lib/python3.9/site-packages/pyvo/dal/query.py:339: DALOverflowWarning: Partial result set. Potential causes MAXREC, async storage space, etc.
  warn("Partial result set. Potential causes MAXREC, async storage space, etc.",


In [6]:
table = eso_instance.query_instrument(instrument) # query the instrument

We can now inspect the results, which are stored as an ``astropy.table``. However note here these results include all data from the science achive, included non-science data... 

In [7]:
table[:3] # print the first 3 rows of the table

access_estsize,access_url,datalink_url,date_obs,dec,det_dit,det_ncorrs_name,det_ndit,dimm_fwhm_avg,dimm_fwhm_rms,dp_cat,dp_id,dp_tech,dp_type,ecl_lat,ecl_lon,exp_start,exposure,exptime,gal_lat,gal_lon,ins_filt1_name,ins_filt2_name,ins_mode,instrument,lambda_max,lambda_min,last_mod_date,lst,mjd_obs,moon_illu,night_flag,ob_container_id,ob_container_parent_id,ob_container_parent_type,ob_container_type,ob_id,ob_name,ob_start,ob_targ_name,ob_tplno,object,observing_mode,origfile,period,pi_coi,prog_id,prog_title,prog_type,ra,release_date,s_region,tel_airm_end,tel_airm_start,tel_alt,tel_ambi_fwhm_end,tel_ambi_fwhm_start,tel_ambi_pres_end,tel_ambi_pres_start,tel_ambi_rhum,tel_az,telescope,tpl_expno,tpl_id,tpl_name,tpl_nexp,tpl_start,utc
kbyte,,,,deg,s,,,arcsec,arcsec,,,,,deg,deg,,s,s,deg,deg,,,,,nm,nm,,s,d,,,,,,,,,,,,,,,,,,,,deg,,,,,deg,arcsec,arcsec,hPa,hPa,%,deg,,,,,,,s
int64,object,object,object,float64,float32,object,int32,float32,float32,object,object,object,object,float64,float64,object,float32,float32,float64,float64,object,object,object,object,float64,float64,object,float32,float64,int16,object,int32,int32,object,object,int32,object,object,object,int32,object,object,object,int32,object,object,object,object,float64,object,object,float32,float32,float32,float32,float32,float32,float32,float32,float32,object,int32,object,object,int32,object,float32
1142,https://dataportal.eso.org/dataPortal/file/HAWKI.2007-07-31T03:00:36.759,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?HAWKI.2007-07-31T03:00:36.759,2007-07-31T03:00:36.7591,--,1.2572,Double,1,1.09,0.31464267,CALIB,HAWKI.2007-07-31T03:00:36.759,IMAGE,STD,--,--,2007-07-31T03:00:36.757Z,1.257,1.257,--,--,Y,OPEN,NOTCS,HAWKI,1070.0,969.0,2011-12-01T17:07:48.690Z,--,54312.12542545,98,night,--,--,,,-1,Test,2007-07-31T03:00:05,,6,STD,Vervice,HAWKI_IMG_STD_GenericOffset212_0013.fits,--,Control Model Accoun,60.A-9282(A),Control Model run for HAWKI,Normal,--,2007-07-31T03:00:36.760Z,,--,--,--,--,--,--,--,--,--,,2,HAWKI_img_cal_GenericOffset,Imaging standards with offsets,11,2007-07-31T03:00:22,--
1139,https://dataportal.eso.org/dataPortal/file/HAWKI.2007-07-31T03:00:43.919,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?HAWKI.2007-07-31T03:00:43.919,2007-07-31T03:00:43.9192,--,1.2572,Double,1,1.09,0.31464267,CALIB,HAWKI.2007-07-31T03:00:43.919,IMAGE,STD,--,--,2007-07-31T03:00:43.917Z,1.257,1.257,--,--,Y,OPEN,NOTCS,HAWKI,1070.0,969.0,2011-12-01T17:08:05.993Z,--,54312.12550832,98,night,--,--,,,-1,Test,2007-07-31T03:00:05,,6,STD,Vervice,HAWKI_IMG_STD_GenericOffset212_0014.fits,--,Control Model Accoun,60.A-9282(A),Control Model run for HAWKI,Normal,--,2007-07-31T03:00:43.920Z,,--,--,--,--,--,--,--,--,--,,3,HAWKI_img_cal_GenericOffset,Imaging standards with offsets,11,2007-07-31T03:00:22,--
1130,https://dataportal.eso.org/dataPortal/file/HAWKI.2007-07-31T03:00:51.081,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?HAWKI.2007-07-31T03:00:51.081,2007-07-31T03:00:51.0802,--,1.2572,Double,1,1.06,0.2521904,CALIB,HAWKI.2007-07-31T03:00:51.081,IMAGE,STD,--,--,2007-07-31T03:00:51.807Z,1.257,1.257,--,--,Y,OPEN,NOTCS,HAWKI,1070.0,969.0,2011-12-01T17:07:50.463Z,--,54312.12559121,98,night,--,--,,,-1,Test,2007-07-31T03:00:05,,6,STD,Vervice,HAWKI_IMG_STD_GenericOffset212_0015.fits,--,Control Model Accoun,60.A-9282(A),Control Model run for HAWKI,Normal,--,2007-07-31T03:00:51.080Z,,--,--,--,--,--,--,--,--,--,,4,HAWKI_img_cal_GenericOffset,Imaging standards with offsets,11,2007-07-31T03:00:22,--


To search for only the science data, we need to refine our results using the following ``column_filters={'dp_cat':'SCIENCE'}``
```python
table = eso.query_instrument(instrument, ra=ra.value, dec=dec.value, radius=radius.value, column_filters={'dp_cat':'SCIENCE'}) 
```
or alternatively we could use the could simply filter the results already in ``table``. 

In [8]:
# table = eso.query_instrument(instrument, ra=ra.value, dec=dec.value, radius=radius.value, column_filters={'dp_cat':'SCIENCE'}) 

In [9]:
table = table[table['dp_cat']=="SCIENCE"] # select only the rows with the dp_cat column equal to "SCIENCE"
table[:3] # print the first 3 rows of the table

access_estsize,access_url,datalink_url,date_obs,dec,det_dit,det_ncorrs_name,det_ndit,dimm_fwhm_avg,dimm_fwhm_rms,dp_cat,dp_id,dp_tech,dp_type,ecl_lat,ecl_lon,exp_start,exposure,exptime,gal_lat,gal_lon,ins_filt1_name,ins_filt2_name,ins_mode,instrument,lambda_max,lambda_min,last_mod_date,lst,mjd_obs,moon_illu,night_flag,ob_container_id,ob_container_parent_id,ob_container_parent_type,ob_container_type,ob_id,ob_name,ob_start,ob_targ_name,ob_tplno,object,observing_mode,origfile,period,pi_coi,prog_id,prog_title,prog_type,ra,release_date,s_region,tel_airm_end,tel_airm_start,tel_alt,tel_ambi_fwhm_end,tel_ambi_fwhm_start,tel_ambi_pres_end,tel_ambi_pres_start,tel_ambi_rhum,tel_az,telescope,tpl_expno,tpl_id,tpl_name,tpl_nexp,tpl_start,utc
kbyte,,,,deg,s,,,arcsec,arcsec,,,,,deg,deg,,s,s,deg,deg,,,,,nm,nm,,s,d,,,,,,,,,,,,,,,,,,,,deg,,,,,deg,arcsec,arcsec,hPa,hPa,%,deg,,,,,,,s
int64,object,object,object,float64,float32,object,int32,float32,float32,object,object,object,object,float64,float64,object,float32,float32,float64,float64,object,object,object,object,float64,float64,object,float32,float64,int16,object,int32,int32,object,object,int32,object,object,object,int32,object,object,object,int32,object,object,object,object,float64,object,object,float32,float32,float32,float32,float32,float32,float32,float32,float32,object,int32,object,object,int32,object,float32
1137,https://dataportal.eso.org/dataPortal/file/HAWKI.2007-07-31T03:01:52.319,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?HAWKI.2007-07-31T03:01:52.319,2007-07-31T03:01:52.3188,--,1.2572,Double,1,1.14,0.39949968,SCIENCE,HAWKI.2007-07-31T03:01:52.319,IMAGE,OBJECT,--,--,2007-07-31T03:01:52.320Z,1.257,1.257,--,--,H,OPEN,IMG,HAWKI,1764.0,1474.0,2011-12-01T17:07:56.687Z,--,54312.12629999,98,night,--,--,,,-1,Test,2007-07-31T03:00:05,,7,,Vervice,HAWKI_IMG_OBS_AutoJitter212_0011.fits,--,Control Model Accoun,60.A-9282(A),Control Model run for HAWKI,Normal,--,2008-07-31T03:01:52.320Z,,--,--,--,--,--,--,--,--,--,ESO-VLT-U4,1,HAWKI_img_obs_AutoJitter,Imaging with jitter (no offsets),10,2007-07-31T03:01:46,--
1185,https://dataportal.eso.org/dataPortal/file/HAWKI.2007-07-31T03:12:51.681,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?HAWKI.2007-07-31T03:12:51.681,2007-07-31T03:12:51.6810,--,1.2572,Double,1,1.3,0.6244998,SCIENCE,HAWKI.2007-07-31T03:12:51.681,IMAGE,OBJECT,--,--,2007-07-31T03:12:51.680Z,1.257,1.257,--,--,H,OPEN,IMG,HAWKI,1764.0,1474.0,2011-12-01T17:08:03.183Z,--,54312.13393149,98,night,--,--,,,-1,Test,2007-07-31T03:11:21,,7,,Vervice,HAWKI_IMG_OBS_AutoJitter212_0012.fits,--,Control Model Accoun,60.A-9282(A),Control Model run for HAWKI,Normal,--,2008-07-31T03:12:51.680Z,,--,--,--,--,--,--,--,--,--,ESO-VLT-U4,1,HAWKI_img_obs_AutoJitter,Imaging with jitter (no offsets),10,2007-07-31T03:12:45,--
1160,https://dataportal.eso.org/dataPortal/file/HAWKI.2007-07-31T03:12:58.842,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?HAWKI.2007-07-31T03:12:58.842,2007-07-31T03:12:58.8423,--,1.2572,Double,1,1.3,0.6244998,SCIENCE,HAWKI.2007-07-31T03:12:58.842,IMAGE,OBJECT,--,--,2007-07-31T03:12:58.843Z,1.257,1.257,--,--,H,OPEN,IMG,HAWKI,1764.0,1474.0,2011-12-01T17:07:50.623Z,--,54312.13401438,98,night,--,--,,,-1,Test,2007-07-31T03:11:21,,7,,Vervice,HAWKI_IMG_OBS_AutoJitter212_0013.fits,--,Control Model Accoun,60.A-9282(A),Control Model run for HAWKI,Normal,--,2008-07-31T03:12:58.843Z,,--,--,--,--,--,--,--,--,--,ESO-VLT-U4,2,HAWKI_img_obs_AutoJitter,Imaging with jitter (no offsets),10,2007-07-31T03:12:45,--


We can check all the column names in the table by printing the ``table.colnames`` attribute to see what else we could filter... 

In [10]:
table.colnames # print the column names of the table

['access_estsize',
 'access_url',
 'datalink_url',
 'date_obs',
 'dec',
 'det_dit',
 'det_ncorrs_name',
 'det_ndit',
 'dimm_fwhm_avg',
 'dimm_fwhm_rms',
 'dp_cat',
 'dp_id',
 'dp_tech',
 'dp_type',
 'ecl_lat',
 'ecl_lon',
 'exp_start',
 'exposure',
 'exptime',
 'gal_lat',
 'gal_lon',
 'ins_filt1_name',
 'ins_filt2_name',
 'ins_mode',
 'instrument',
 'lambda_max',
 'lambda_min',
 'last_mod_date',
 'lst',
 'mjd_obs',
 'moon_illu',
 'night_flag',
 'ob_container_id',
 'ob_container_parent_id',
 'ob_container_parent_type',
 'ob_container_type',
 'ob_id',
 'ob_name',
 'ob_start',
 'ob_targ_name',
 'ob_tplno',
 'object',
 'observing_mode',
 'origfile',
 'period',
 'pi_coi',
 'prog_id',
 'prog_title',
 'prog_type',
 'ra',
 'release_date',
 's_region',
 'tel_airm_end',
 'tel_airm_start',
 'tel_alt',
 'tel_ambi_fwhm_end',
 'tel_ambi_fwhm_start',
 'tel_ambi_pres_end',
 'tel_ambi_pres_start',
 'tel_ambi_rhum',
 'tel_az',
 'telescope',
 'tpl_expno',
 'tpl_id',
 'tpl_name',
 'tpl_nexp',
 'tpl_start'

## Downloading datasets from the archive

The returned table has an ``dp_id`` column. It can be used to retrieve the datasets with ``retrieve_data()``.

The file names, returned in ``data_files``, points to the decompressed datasets (without the .Z extension) that have been locally downloaded. 

The default location (in the ``astropy`` cache) of the decompressed datasets can be adjusted by providing a destination keyword in the call to ``retrieve_data()``. By default, if a requested dataset is already found, it is not downloaded again from the archive. To force the retrieval of data that are present in the destination directory, use ``continuation=True`` in the call to ``retrieve_data()``.

In [11]:
data_files = eso_instance.retrieve_data(table['dp_id'][0]) # download the first data product
# data_files = eso.retrieve_data(table['dp_id'][0], continuation=True) # force the download of all files even if present in the destination directory
# data_files = eso.retrieve_data(table['dp_id'][0], destination='./') # specify the destination directory for the download

INFO: Downloading datasets ... [astroquery.eso.core]
INFO: Downloading 1 files ... [astroquery.eso.core]
INFO: Downloading file 1/1 https://dataportal.eso.org/dataPortal/file/HAWKI.2007-07-31T03:01:52.319 to /Users/jcarmona/.astropy/cache/astroquery/Eso [astroquery.eso.core]
INFO: Done! [astroquery.eso.core]


ERROR: Access denied to https://dataportal.eso.org/dataPortal/file/HAWKI.2007-07-31T03:01:52.319 [astroquery.eso.core]


<hr style="border:2px solid #0281c9"> </hr>